# **Who Made the Tackle**

Thanks to the [nflfastR](https://www.nflfastr.com/) project, play by play data is available for seasons dating back to 1999. 
The data they have available contains significant information that can be derived but is not currently available in the Big Data Bowl dataset for this year's competition. When looking through the data I noticed there was no specific column for which defender made the tackle on a given play. Rather than parse out the names of the defenders who made the tackle from the play description, the nflfastR play-by-play data for 2018 conatins several colums of tackle information including solo tackle player names, and assist tackle player names, etc. In this notebook you'll find some code to join the nflfastR data to the competition data. 

The nflfastR data files and links to more info can be found on kaggle [here](https://www.kaggle.com/benbaldwin/nflfastr).

I am not proficient in Python and was having trouble decoding the player IDs in the nflfastR file in R, so I took the code from [this notebook](https://www.kaggle.com/s903124/getting-passer-and-receiver-identity-from-nfl-data) and adapted for what I needed. 

I was interested in only the solo_tackle_1_player, assist_tackle_1_player, and assist_tackle_2_player but you can put in any nflfastR ID column that has the new long format ID and make sure to just replace the variable names in the code.

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_columns = 999
import codecs
decode_hex = codecs.getdecoder("hex_codec")

In [ ]:
# Get roster data from nflfastR github repo
roster_data = pd.read_csv('https://raw.githubusercontent.com/guga31bb/nflfastR-data/master/roster-data/roster.csv', low_memory = False)
roster_data = roster_data[['teamPlayers.gsisId','teamPlayers.nflId']].drop_duplicates().dropna()
roster_data.head()

In [ ]:
# Get nflfastR play-by-play data from Kaggle page , encoding = 'unicode_escape'
fastr_18 = pd.read_csv('../input/nflfastr/play_by_play_2018.csv')

# Convert new ID format to gsis ID
fastr_18['solo_tackler1_gsis_id'] =  (fastr_18['solo_tackle_1_player_id'].str.split('-').str[2].str[-2:] + fastr_18['solo_tackle_1_player_id'].str.split('-').str[3] + fastr_18['solo_tackle_1_player_id'].str.split('-').str[4].str[:4]).apply(lambda x: decode_hex(x)[0].decode("utf-8") if(pd.notnull(x))  else x )
fastr_18['solo_tackler1_gsis_id'] ='00-' +  fastr_18.loc[~pd.isna(fastr_18['solo_tackler1_gsis_id']), 'solo_tackler1_gsis_id'].astype(str).str.zfill(7)

fastr_18['assist_tackler1_gsis_id'] =  (fastr_18['assist_tackle_1_player_id'].str.split('-').str[2].str[-2:] + fastr_18['assist_tackle_1_player_id'].str.split('-').str[3] + fastr_18['assist_tackle_1_player_id'].str.split('-').str[4].str[:4]).apply(lambda x: decode_hex(x)[0].decode("utf-8") if(pd.notnull(x))  else x )
fastr_18['assist_tackler1_gsis_id'] ='00-' +  fastr_18.loc[~pd.isna(fastr_18['assist_tackler1_gsis_id']), 'assist_tackler1_gsis_id'].astype(str).str.zfill(7)

fastr_18['assist_tackler2_gsis_id'] =  (fastr_18['assist_tackle_2_player_id'].str.split('-').str[2].str[-2:] + fastr_18['assist_tackle_1_player_id'].str.split('-').str[3] + fastr_18['assist_tackle_1_player_id'].str.split('-').str[4].str[:4]).apply(lambda x: decode_hex(x)[0].decode("utf-8") if(pd.notnull(x))  else x )
fastr_18['assist_tackler2_gsis_id'] ='00-' +  fastr_18.loc[~pd.isna(fastr_18['assist_tackler2_gsis_id']), 'assist_tackler2_gsis_id'].astype(str).str.zfill(7)

#join to roster data
fastr_18['solo1_nflId'] =  pd.merge(fastr_18[['solo_tackler1_gsis_id' ]],roster_data[['teamPlayers.nflId','teamPlayers.gsisId']].dropna(),left_on='solo_tackler1_gsis_id',right_on='teamPlayers.gsisId',how='left')['teamPlayers.nflId']
fastr_18['assist1_nflId'] =  pd.merge(fastr_18[['assist_tackler1_gsis_id' ]],roster_data[['teamPlayers.nflId','teamPlayers.gsisId']].dropna(),left_on='assist_tackler1_gsis_id',right_on='teamPlayers.gsisId',how='left')['teamPlayers.nflId']
fastr_18['assist2_nflId'] =  pd.merge(fastr_18[['assist_tackler2_gsis_id' ]],roster_data[['teamPlayers.nflId','teamPlayers.gsisId']].dropna(),left_on='assist_tackler2_gsis_id',right_on='teamPlayers.gsisId',how='left')['teamPlayers.nflId']

fastr_18.head()

Now that you have converted the ID for the desired colums you can join the nflfastR file to the competition data using gameId and playId as your primary keys in the competition data, and old_game_id and play_id as your keys for the nflfastR data.

In [ ]:
week1_df = pd.read_csv('../input/nfl-big-data-bowl-2021/week1.csv')
week1_df.head()

In [ ]:
week1_df = pd.merge(week1_df,fastr_18[['play_id', 'old_game_id','solo1_nflId','assist1_nflId', 'assist2_nflId']],left_on=['gameId','playId'],right_on=['old_game_id','play_id'],how='left')

In [ ]:
week1_df.tail()